# 텍스트 분석
***
## Bag of Words = BOW

#### Bag of Words는 문서내 모든 단어를 문맥이나 순서와 무관하게 단어에 대해 빈도 값을 부여해 feature를 추출하는 방식이다. 간단히 설명하면 아래 방식과 같다.

In [61]:
text1 = "He likes apple"
text2 = "Apple is a fruit i love but apple is expensive"
text3 = "I love apple and banana"

In [62]:
words1 = list(map(lambda x : x.lower(), text1.split(' ')))
words2 = list(map(lambda x : x.lower(), text2.split(' ')))
words3 = list(map(lambda x : x.lower(), text3.split(' ')))

In [63]:
words1

['he', 'likes', 'apple']

In [64]:
words2

['apple', 'is', 'a', 'fruit', 'i', 'love', 'but', 'apple', 'is', 'expensive']

In [65]:
words3

['i', 'love', 'apple', 'and', 'banana']

In [66]:
import pandas as pd

In [67]:
bow = pd.DataFrame(index = ['문장1', '문장2', '문장3'], columns = list(set(words1 + words2 + words3))).fillna(0)

In [68]:
for i in range(1, 4) :
    for feature in bow.columns :
        for w in globals()[f'words{i}'] :
            if w == feature :
                bow.loc[f'문장{i}', feature] += 1
            else :
                pass

In [69]:
bow

,is,fruit,and,a,likes,apple,love,i,banana,but,expensive,he
문장1,0,0,0,0,1,1,0,0,0,0,0,1
문장2,2,1,0,1,0,2,1,1,0,1,1,0
문장3,0,0,1,0,0,1,1,1,1,0,0,0


각 문장 내 단어들에 대해 문장에서 등장하는 빈도를 값을 가지는 테이블을 생성한다.

2번 문장의 경우 applea과 is가 2번씩 등장하므로 2의 값을 가진다.

이처럼 BOW는 쉽고 빠르게 구축이 가능하다. 하지만 치명적인 한계 또한 존재하는데 대표적인 단점은 아래와 같다.

**- 문맥 의미 반영 부족** : 단어의 순서를 고려하지 않기 떄문에 문맥적인 의미를 고려할 수 없다. 이를 보완하기 위해 n-gram을 사용하긴 하지만 완전히 해소하진 못한다.

**- 희소 행렬 문제** : 문서 혹은 문장이 매우 많은 경우 BOW를 적용하면 차원이 굉장히 커진다. 생성된 feature가 100만개 이지만 n번째 문서 혹은 문장에서 사용된 단어는 3개라면 999만9997개 차원의 값이 0으로 채워진다. 이렇게 될 경우 머신러닝 적용할 때 시간과 예측 성능 모두 안좋아 질 수 있다.

***

BOW의 단점을 보완하기 위해 **feature vectorization**을 사용하는데 크게 두 가지 방식이 존재한다.

**- Count 기반** : word feature에 각 문서에서 해당 단어의 빈도 즉 count를 부여하는 방식. 단순히 빈도를 의미하기 때문에 stopwords와 같이 문장에서 자연스럽게 자주 쓰일 수 밖에 없는 단어들까지도 높은 빈도 값을 부여한다.

**- TF-IDF** : 개별 문서에서 자주 나타나는 단어는 높은 가중치를, 모든 문서에서 전반적으로 나타나는 단어에 대해서는 페널티를 주는 방식.
***
Sklearn에 CountVectorizer와 TfidfVectorizer를 사용하면 위의 두 방식을 활용할 수 있다.

**- CountVectorizer parameter**(Tfidf도 동일)

|Parameter|description|
|---|---|
|max_df|최대 빈도 수, 과하게 반복되는 단어를 배제할 수 있다.|
|min_df|최소 빈도 수, 과하게 빈도가 낮은 단어를 배제한다.|
|max_features|추출하는 feature의 개수 제한, 상위 n개까지만 추출|
|stop_words|스톱 워드 사용|
|n_gram_range|n_gram범위를 정한다. 튜플 형태로 지정|
|analyzer|feature 추출 수행 단위 지정, 디폴트는 word로 charcter 단위도 가능하다.|
|token_pattern|토큰화하는 정규 표현식 패턴, analyzer = True일 때만 변경 가능|
|tokenizer|토큰화를 별도의 커스텀으로 지정할 수 있다.|
***

### 희소행렬 - COO 형식

COO형식은 0이 아닌 데이터만 별도의 array에 저장하고 각 데이터 값이 가리키는 행과 열의 위치를 별도의 배열로 저장하는 방식이다.

배열에서 0이 아닌 값들의 위치 정보를 (행, 열)로 변환하고 행의 값들과 열의 값들을 별도로 지정한 후 희소 행렬을 생성한다.

In [1]:
import numpy as np

In [71]:
dense = np.array([[3, 0, 1], [0, 2, 0]])

In [2]:
from scipy import sparse

In [73]:
data = np.array([3, 1, 2])

In [74]:
r = np.array([0, 0, 1])
c = np.array([0, 2, 1])

In [76]:
coo = sparse.coo_matrix((data, (r, c)))

In [78]:
coo.toarray()

array([[3, 0, 1],
       [0, 2, 0]])

### 희소행렬 - CSR 형식

CSR형식은 COO의 문제점을 해결한 것으로 메모리가 적게 들고 빠른 연산이 가능하다.

행 위치 배열 내에 있는 고유한 값의 시작 위치만 다시 별도의 위치 배열로 가지는 변환 방식을 의미한다.

In [3]:
dense = np.array([[0, 0, 1, 0, 0, 5],
                 [1, 4, 0, 3, 2, 5],
                 [0, 6, 0, 3, 0, 0],
                 [2, 0, 0, 0, 0, 0],
                 [0, 0, 0, 7, 0, 8],
                 [1, 0, 0, 0, 0, 0]])

In [5]:
# 0값이 아닌 데이터 추출
not_0 = np.array([1, 5, 1, 4, 3, 2, 5, 6, 3, 2, 7, 8, 1])

In [6]:
r_pos = np.array([0, 0, 1, 1, 1, 1, 1, 2, 2, 3, 4, 4, 5]) # 0이 아닌 값이 등장한 행의 인덱스
c_pos = np.array([2, 5, 0, 1, 3, 4, 5, 1, 3, 0, 3, 5, 0]) # 0이 아닌 값이 등장한 열의 인덱스 

In [8]:
# COO형식
COO = sparse.coo_matrix((not_0, (r_pos, c_pos)))

In [10]:
# [0, 0, 1, 1, 1, 1, 1, 2, 2, 3, 4, 4, 5]에서 행 위치 배열의 인덱스는 고유한 값이 새로 등장한 인덱스
# 즉, 0, 1, 2, 3, 4, 5가 처음 등장한 인덱스를 의미한다.
# 따라서 [0, 2, 7, 9, 10, 12]가 되고 맨 끝에 총 항목 개수인 13을 추가한다.

# 행 위치 배열의 고유 값의 시작 위치 인덱스를 배열로 생성
r_pos_idx = np.array([0, 2, 7, 9, 10, 12, 13])

In [12]:
# CSR형식
CSR = sparse.csr_matrix((not_0, c_pos, r_pos_idx))

#### COO 형식 원래 배열로 변환

In [13]:
print(COO.toarray())

[[0 0 1 0 0 5]
 [1 4 0 3 2 5]
 [0 6 0 3 0 0]
 [2 0 0 0 0 0]
 [0 0 0 7 0 8]
 [1 0 0 0 0 0]]


#### CSR 형식 원래 배열로 변환

In [14]:
print(CSR.toarray())

[[0 0 1 0 0 5]
 [1 4 0 3 2 5]
 [0 6 0 3 0 0]
 [2 0 0 0 0 0]
 [0 0 0 7 0 8]
 [1 0 0 0 0 0]]


실제 사용시에는 행렬을 파라미터로 사용하면 COO나 CSR 희소 행렬로 생성된다.

In [15]:
coo = sparse.coo_matrix(dense)

In [17]:
coo.toarray()

array([[0, 0, 1, 0, 0, 5],
       [1, 4, 0, 3, 2, 5],
       [0, 6, 0, 3, 0, 0],
       [2, 0, 0, 0, 0, 0],
       [0, 0, 0, 7, 0, 8],
       [1, 0, 0, 0, 0, 0]])

In [18]:
csr = sparse.coo_matrix(dense)

In [19]:
csr.toarray()

array([[0, 0, 1, 0, 0, 5],
       [1, 4, 0, 3, 2, 5],
       [0, 6, 0, 3, 0, 0],
       [2, 0, 0, 0, 0, 0],
       [0, 0, 0, 7, 0, 8],
       [1, 0, 0, 0, 0, 0]])

sklearn의 CountVectorizer나 TfidfVectorizer 클래스로 변환된 feature는 모두 CSR 형태의 희소 행렬이다.